In [173]:
import pandas as pd
import numpy as np
import os
import datetime

In [174]:
wkdir = "/Users/chrisolen/Documents/uchicago_courses/deep_learning_and_image_recognition/finance/fin_portfolio_optimization/"
data_files = os.listdir(wkdir+'data')
data_files.remove('.DS_Store')

In [175]:
data_dict = {'UST_10YR':'10-Year US Treasuries (Daily)', 'UIVE_SP500VALUEETF':'SP500 iShares Value ETF (Daily)',
            'VNQ_VANGREALEST':'Vanguard Real Estate ETF (Daily)','USFFR':'US Fed Funds Rate (Daily)',
            'EMB_USDEMRGBOND':'Emerging Markets Bond Fund (Daily)','LQD_CORPBOND':'Corporate Bond Fund (Daily)',
            'MUB_MUNIBOND':'Municipal Bond Fund (Daily)','CHNGDP':'Chinese GDP (Quarterly)',
            'SHY_1-3USTR':'1-3-Year US Treasury Bonds (Daily)', 'USDJPY':'USD-JPY FX Rate (Daily)',
            'USDGBP':'USD-GBP FX Rate (Daily)','VIG_VANGDIV':'Vanguard Dividend Growth Fund (Daily)',
            'IVV_SP500':'Core SP500 ETF Market-Cap-Weighted (Daily)','USDRMB':'USD-RMB FX Rate (Daily)',
            'CRUDOIL':'Crude Oil Futures (Daily)','CFE_VIX':'Volatility Index (Daily)',
            'EEM_MSCIEMERGING':'Emerging Market Index Fund (Daily)','US_UNEMP':'US Unemployment Rate (Monthly)',
            'USDEUR':'USD-EUR FX Rate (Daily)','USGDP':'US GDP (Quarterly)',
            'XLE_ENERGYSPDR':'Energy Companies Index Fund (Daily)', 'SP500_GSCI':'Commodity Market Index Fund (Daily)',
            'EZGDP':'Eurozone GDP (Daily)','EFA_MSCIEAFE':'Developed Equity Markets Index Fund',
            'TIP_TIPSBOND':'Treasury Bond Indexed to Inflation (Daily)','UST_2YR':'2-Year US Treasury Bonds',
            'USDOIS':'Overnight Indexed Swap'}

In [ ]:
# Separating features into macroeconomic indictors and portfolio:

econ = ['CHNGDP','USGDP','EZGDP','US_UNEMP']

finstruments = ['UST_10YR','USFFR','USDRMB','CRUDOIL','CFE_VIX','USDEUR','UST_2YR',
             'SP500_GSCI','USDOIS','UIVE_SP500VALUEETF','USDJPY','USDGBP']

assets = ['VNQ_VANGREALEST','EMB_USDEMRGBOND','LQD_CORPBOND',
            'MUB_MUNIBOND','SHY_1-3USTR','VIG_VANGDIV','IVV_SP500','EEM_MSCIEMERGING',
            'XLE_ENERGYSPDR','EFA_MSCIEAFE','TIP_TIPSBOND']

In [176]:
# Loading time series from file into a list of dataframes:

df_list = []

for i in range(len(data_files)):
    temp_df = pd.read_csv(wkdir+'data/'+ data_files[i], encoding = 'unicode_escape', header=None ).iloc[6:,]
    df_list.append(temp_df)


In [177]:
# Determining numbers of each time unit:

time_units = []

for i in range(len(df_list)):
    time_units.append(df_list[i].shape[0])    
    
time_units = set(time_units) # pull out unique avlues
time_units = list(time_units) # back to list for indexing
time_units = sorted(time_units)  # and sorting
time_units

n_days = time_units[-1] # assumes daily is the most granular
n_weeks = []
n_months = []
n_quarters = []

for i in range(len(time_units[0:-1])):
    if n_days/time_units[i] <= 10:
        n_weeks = time_units[i]
    elif n_days/time_units[i] > 10 and n_days/time_units[i] <= 40:
        n_months = time_units[i]
    else:
        n_quarters = time_units[i]
        
        
    

In [178]:
# Cleaning and merging time series:

first_daily_index = [i for i in range(len(df_list)) if len(df_list[i]) == n_days][0]
dailys = df_list[first_daily_index] # creation of new df of daily figures
dailys.rename({0:'date'}, axis=1, inplace=True) # renaming first column of first dataframe

quarterlys = pd.DataFrame() 
monthlys = pd.DataFrame()

counter_dailys = 0
counter_quarterlys = 0
counter_monthlys = 0


for i in range(1,len(df_list)):
    
    # reformatting if the date string is smaller than m/d/yyyy
    if len(df_list[i][0].iloc[0]) < 8: 
        df_list[i][0]=[datetime.datetime.strptime(df_list[i][0].iloc[j], "%m/%d/%y").strftime('%-m/%-d/%Y') for j in range(len(df_list[i]))]
    else:
        pass
    
    # for daily time series
    if len(df_list[i])==n_days: 
        counter_dailys += 1
        dailys = dailys.merge(df_list[i], left_on=dailys['date'], right_on=df_list[i][0])
        dailys.drop(['key_0',0], axis=1, inplace=True)
        dailys.rename(columns={dailys.columns[1+counter_dailys]:data_files[i][:-4]}, inplace=True)
        print("iteration {} is done".format(i))
        
    # for quarterly time series
    elif len(df_list[i])==n_quarters: 
        
        # keeping both date and values for the first one
        if quarterlys.shape[0]==0: 
            counter_quarterlys += 1
            df_list[i].rename({0:'date'}, axis=1, inplace=True)
            quarterlys = pd.concat([quarterlys,df_list[i]], axis=1)
            quarterlys.rename(columns={quarterlys.columns[counter_quarterlys]:data_files[i][:-4]}, inplace=True)
            print("iteration {} is done".format(i))
        
        # keeping only values for the rest
        else: 
            counter_quarterlys += 1
            quarterlys = pd.concat([quarterlys,df_list[i].iloc[:,1]], axis=1)
            quarterlys.rename(columns={quarterlys.columns[counter_quarterlys]:data_files[i][:-4]}, inplace=True)
            print("iteration {} is done".format(i))
    
    # for monthly time series        
    else: 
        counter_monthlys += 1
        df_list[i].rename({0:'date'}, axis=1, inplace=True)
        monthlys = pd.concat([monthlys,df_list[i]], axis=1)
        monthlys.rename(columns={monthlys.columns[counter_monthlys]:data_files[i][:-4]}, inplace=True)
        print("iteration {} is done".format(i))
        
# now just need to rename the first column
dailys.rename(columns={dailys.columns[1]:data_files[0][:-4]}, inplace=True) 

# resetting indexes with date
dailys.set_index('date', drop = True, inplace=True)
quarterlys.set_index('date', drop = True, inplace=True)
monthlys.set_index('date', drop = True, inplace=True)

# joining dfs
first_join = dailys.join(quarterlys, how = 'left')
data = first_join.join(monthlys, how = 'left')
    

iteration 1 is done
iteration 2 is done
iteration 3 is done
iteration 4 is done
iteration 5 is done
iteration 6 is done
iteration 7 is done
iteration 8 is done
iteration 9 is done
iteration 10 is done
iteration 11 is done
iteration 12 is done
iteration 13 is done
iteration 14 is done
iteration 15 is done
iteration 16 is done
iteration 17 is done
iteration 18 is done
iteration 19 is done
iteration 20 is done
iteration 21 is done
iteration 22 is done
iteration 23 is done
iteration 24 is done
iteration 25 is done
iteration 26 is done


In [179]:
# Given data availability, we begin in 2008:

data = data.loc['1/2/2008':,:] 

In [180]:
data.tail()

,UST_10YR,UIVE_SP500VALUEETF,VNQ_VANGREALEST,USFFR,EMB_USDEMRGBOND,LQD_CORPBOND,MUB_MUNIBOND,SHY_1-3USTR,USDJPY,USDGBP,...,XLE_ENERGYSPDR,SP500_GSCI,EFA_MSCIEAFE,TIP_TIPSBOND,UST_2YR,USDOIS,CHNGDP,USGDP,EZGDP,US_UNEMP
date,,,,,,,,,,,,,,,,,,,,,
10/2/2019,1.5969,116.89,91.98,1.85,112.39,127.24,114.13,84.87,0.933,1.23015,...,56.41,397.184,25.61,116.32,1.483,1.836,NaN,NaN,NaN,NaN
10/3/2019,1.5353,115.31,92.87,1.83,112.93,127.74,114.37,85.05,0.9353499,1.23295,...,57.02,397.356,25.73,116.69,1.3871,1.808,NaN,NaN,NaN,NaN
10/4/2019,1.5131,116.48,93.43,1.82,113.92,128.27,114.37,85.04,0.93505,1.23335,...,57.38,399.469,25.92,117.05,1.3947,1.798,NaN,NaN,NaN,NaN
10/7/2019,1.5523,117.46,93.23,1.82,113.28,127.62,114.28,84.95,0.932,1.2292,...,56.87,399.211,25.97,116.73,1.4668,1.777,NaN,NaN,NaN,NaN
10/8/2019,1.5386,116.35,92.74,1.82,113.17,127.51,114.61,85,0.93385,1.2219,...,55.9,399.49,25.75,116.62,1.4265,1.769,NaN,NaN,NaN,NaN


In [181]:
data.shape

(3070, 27)

In [182]:
# Converting everything from string to float:

for i in range(len(data.columns)):
    data.loc[:,data.columns[i]] = data.loc[:,data.columns[i]].astype(float)
    

In [183]:
data.dtypes

UST_10YR              float64
UIVE_SP500VALUEETF    float64
VNQ_VANGREALEST       float64
USFFR                 float64
EMB_USDEMRGBOND       float64
LQD_CORPBOND          float64
MUB_MUNIBOND          float64
SHY_1-3USTR           float64
USDJPY                float64
USDGBP                float64
VIG_VANGDIV           float64
IVV_SP500             float64
USDRMB                float64
CRUDOIL               float64
CFE_VIX               float64
EEM_MSCIEMERGING      float64
USDEUR                float64
XLE_ENERGYSPDR        float64
SP500_GSCI            float64
EFA_MSCIEAFE          float64
TIP_TIPSBOND          float64
UST_2YR               float64
USDOIS                float64
CHNGDP                float64
USGDP                 float64
EZGDP                 float64
US_UNEMP              float64
dtype: object

In [184]:
# Creating macroeconomic 'shock' variables:

shock = ['CHNGDP_Shock','USGDP_Shock','EZGDP_Shock','US_UNEMP_Shock']
data[shock] = data.loc[:,shock].fillna(0)

In [185]:
# Filling in Nans from macroeconomic variables:

list_for_backfills = ['CHNGDP','USGDP','EZGDP','US_UNEMP']
for i in range(len(list_for_backfills)):
    data.loc[:,list_for_backfills[i]].fillna(method='ffill', inplace=True) # propagating last valid value before series of nans forward to fillna
    data.loc[:,list_for_backfills[i]].fillna(method='bfill', inplace=True) # propagating first valid value back to fillnas


In [186]:
data.head()

,UST_10YR,UIVE_SP500VALUEETF,VNQ_VANGREALEST,USFFR,EMB_USDEMRGBOND,LQD_CORPBOND,MUB_MUNIBOND,SHY_1-3USTR,USDJPY,USDGBP,...,UST_2YR,USDOIS,CHNGDP,USGDP,EZGDP,US_UNEMP,CHNGDP_Shock,USGDP_Shock,EZGDP_Shock,US_UNEMP_Shock
date,,,,,,,,,,,,,,,,,,,,,
1/2/2008,3.8971,76.39,61.27,4.11,101.490,106.08,102.28,82.3401,0.91129,1.98135,...,2.8658,4.139,85399.6,15671.383,2616.0,5.0,0.0,0.0,0.0,0.0
1/3/2008,3.9045,75.51,59.36,4.25,101.670,105.73,102.16,82.4700,0.91320,1.97257,...,2.8248,4.165,85399.6,15671.383,2616.0,5.0,0.0,0.0,0.0,0.0
1/4/2008,3.8542,74.50,56.95,4.18,101.720,106.60,102.11,82.5900,0.92056,1.97325,...,2.7336,4.137,85399.6,15671.383,2616.0,5.0,0.0,0.0,0.0,0.0
1/7/2008,3.8372,73.98,57.48,4.27,101.950,106.47,102.00,82.5400,0.91537,1.96882,...,2.7655,4.071,85399.6,15671.383,2616.0,5.0,0.0,0.0,0.0,0.0
1/8/2008,3.8389,74.28,55.49,4.27,102.197,106.27,101.85,82.6600,0.91798,1.97279,...,2.7607,4.059,85399.6,15671.383,2616.0,5.0,0.0,0.0,0.0,0.0


In [193]:
# Creating volatility features for portfolio assets:

for i in range(len(assets)):
    data = data.join(data[assets].iloc[:,i].rolling(5).std(), how = 'left', rsuffix = '_')
    data.rename(columns={data.columns[-1:][0]:assets[i][0:3]+"_VOL"}, inplace = True)
    
asset_vols = data.columns[-11:]


In [194]:
data.shape

(3070, 42)

In [195]:
data.dropna(inplace=True)

In [196]:
data.shape

(2959, 42)

In [197]:
data.columns

Index(['UST_10YR', 'UIVE_SP500VALUEETF', 'VNQ_VANGREALEST', 'USFFR',
       'EMB_USDEMRGBOND', 'LQD_CORPBOND', 'MUB_MUNIBOND', 'SHY_1-3USTR',
       'USDJPY', 'USDGBP', 'VIG_VANGDIV', 'IVV_SP500', 'USDRMB', 'CRUDOIL',
       'CFE_VIX', 'EEM_MSCIEMERGING', 'USDEUR', 'XLE_ENERGYSPDR', 'SP500_GSCI',
       'EFA_MSCIEAFE', 'TIP_TIPSBOND', 'UST_2YR', 'USDOIS', 'CHNGDP', 'USGDP',
       'EZGDP', 'US_UNEMP', 'CHNGDP_Shock', 'USGDP_Shock', 'EZGDP_Shock',
       'US_UNEMP_Shock', 'VNQ_VOL', 'EMB_VOL', 'LQD_VOL', 'MUB_VOL', 'VIG_VOL',
       'IVV_VOL', 'EEM_VOL', 'EFA_VOL', 'XLE_VOL', 'SHY_VOL', 'TIP_VOL'],
      dtype='object')

In [198]:
data.to_csv(wkdir+'data/'+'data_cleaned.csv')